# Nonlinear analysis -Assignment 1

26.09.2023 - Jérémie Engler and Laure Toullier

## Exercice 1

### Imports

In [1]:
import numpy as np

Definitions

In [8]:
# Element connectivity
nelements = 2 
nnodes = 3

# Written input:
connectivity = np.array([[1, 2], [2, 3]])
nodes_P = np.array([2])

# Unit stiffness matrix of bar in local coordinate system
K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])

# Input data and element stiffness matrices
E = [200e3,200e3]  # N/mm^2 Young modulus of the elements
A = [10e3,15e3] #mm^2 Cross-sectional area of the elements
a = 3000 #mm Half lenght of the truss
b = 4000 #mm Height of the truss
P = 150e3 #N Load
phiP = 30 #° Angle of load
theta = [53.13, -53.13] #° Angle of elements, at first node of the element
Pglob = np.array([P*np.cos(np.radians(phiP)), P*np.sin(np.radians(phiP))]) #N P force in global coordinates (x,y)

Stiffness

In [9]:
T = np.zeros((nelements,4,4))
L = []
k = []
k_glob = np.zeros((nelements,4,4))

for i in range(nelements):
    c = np.cos(np.radians(theta[i]))
    s = np.sin(np.radians(theta[i]))
    Li = abs(a/c)
    if theta[i] == -90:
        Li = b
    Ai = A[i]
    Ei = E[i]
    ki = Ei * Ai / Li
    T[i] = np.array([[c, s, 0, 0], 
              [-s, c, 0, 0], 
              [0, 0, c, s], 
              [0, 0, -s, c]])
    L.append(Li)
    k.append(ki)

for i in range(nelements):
    k_glob[i] = k[i] * T[i].T @ K_unit @ T[i]


# Assemble global stiffness matrix
NDoF = 2*nnodes
K_global = np.zeros((NDoF,NDoF))
K_globals = np.zeros((nelements,NDoF, NDoF))


# Assemble elements
for i in range (nelements):
    connectelem = connectivity[i] #nodes connected to the element
    indexs = 2*connectelem
    i11 = indexs[0]-2
    i12 = indexs[0]
    i21 = indexs[1]-2
    i22 = indexs[1]

    Ki_global = np.zeros((NDoF, NDoF))
    ki_global = k_glob[i]
    Ki_global[i11:i12, i11:i12] = ki_global[0:2, 0:2]
    Ki_global[i21:i22, i11:i12] = ki_global[2:4, 0:2]
    Ki_global[i11:i12, i21:i22] = ki_global[0:2, 2:4]
    Ki_global[i21:i22, i21:i22] = ki_global[2:4, 2:4]
    K_global = K_global + Ki_global
    K_globals[i] = Ki_global

System equilibrium

In [10]:
# Calculate displacements at nodes with forces applied
numnodesP = len(nodes_P)
F_red = Pglob
indexssup = 2*nodes_P - 1
indexsinf = indexssup - 1
mask = []
for i in range(numnodesP):
    mask.append(indexsinf[i])
    mask.append(indexssup[i])
K_red = K_global[mask,:]
K_red = K_red[:,mask]

u_red = np.linalg.inv(K_red) @ F_red


# Calculate reaction forces at other nodes
u_vec = np.zeros((2*nnodes))

for i in range(numnodesP):
    u_vec[indexsinf[i]] = u_red[2*i]
    u_vec[indexssup[i]] = u_red[2*i+1]

F = K_global @ u_vec


# Calculate axial forces
axialF = []
for i in range(nelements):
    F_elei = K_globals[i] @ u_vec
    index_red = 2*connectivity[i]
    F_elei_red = F_elei[[index_red[0]-2, index_red[0]-1, index_red[1]-2, index_red[1]-1]]
    F_elei_local = T[i] @ F_elei_red
    Fi = F_elei_local[2]
    axialF.append(Fi)

for i in range(nelements):
    print(f"Axial load in element {i+1}: {round(axialF[i])} N")
    print()

for i in range(nnodes):
    if i == 0:
        print("Left support:")
    if i == 1:
        print("Right support:")
    print(f"Node {i+1}, horizontal displacement: {round(u_vec[2*i],2)} mm")
    print(f"Node {i+1}, vertical displacement: {round(u_vec[2*i+1],2)} mm")
    print()
    print(f"Node {i+1}, horizontal force: {round(F[2*i])} N")
    print(f"Node {i+1}, vertical force: {round(F[2*i+1])} N")
    print()
    print()

Axial load in element 1: 155128 N

Axial load in element 2: -61378 N

Left support:
Node 1, horizontal displacement: 0.0 mm
Node 1, vertical displacement: 0.0 mm

Node 1, horizontal force: -93077 N
Node 1, vertical force: -124102 N


Right support:
Node 2, horizontal displacement: 0.41 mm
Node 2, vertical displacement: 0.18 mm

Node 2, horizontal force: 129904 N
Node 2, vertical force: 75000 N


Node 3, horizontal displacement: 0.0 mm
Node 3, vertical displacement: 0.0 mm

Node 3, horizontal force: -36827 N
Node 3, vertical force: 49102 N


